In [133]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from ast import literal_eval
from html.parser import HTMLParser
import pickle
from nltk import sent_tokenize, word_tokenize
import random
from collections import Counter

tqdm().pandas()

# read XML file and put into pd df

In [84]:
acrofile = 'data/acrolinx.WordPress.2019-05-21.xml'

In [85]:
tree = ET.parse(acrofile)
root = tree.getroot()
channel = tree.find('channel')

In [86]:
titles = []
dates = []
categories = []
texts = []

for item in channel:
    if item.tag == 'item':
        titles.append(item.find('title').text)
        dates.append(item.find('pubDate').text)
        
        texts.append(item.find('{http://purl.org/rss/1.0/modules/content/}encoded').text)
        
        cats = []
        for child in item:
            if child.tag == 'category':
                cats.append(child.text)
        categories.append(list(set(cats)))

In [87]:
assert len(titles) == len(dates) == len(categories) == len(texts)

In [88]:
df = pd.DataFrame({'title': titles, 
                  'pubdate': dates,
                  'cats': categories,
                  'text': texts})

In [89]:
df.head()

,cats,pubdate,text,title
0,"[Content Marketing, Tech Comm, Blog]","Thu, 20 Nov 2014 12:49:29 +0000",Imagine that you’ve just written what you beli...,10 Questions You’ve Got to Ask Yourself Before...
1,"[Content Marketing, Tech Comm, Love of Languag...","Mon, 17 Nov 2014 12:51:04 +0000","We recently shared <a href=""https://www.acroli...",Problem Words that Give Writers Trouble (Part 2)
2,"[Content Marketing, Tech Comm, Love of Languag...","Thu, 13 Nov 2014 12:52:27 +0000",Even the best writers can get tripped up somet...,Problem Words that Give Writers Trouble (Part 1)
3,"[Content Marketing, Tech Comm, Love of Languag...","Mon, 10 Nov 2014 12:55:17 +0000",Part of being a good content marketer is being...,6 Great Free (or Nearly Free) Tools for Conten...
4,"[Content Marketing, Tech Comm, Love of Languag...","Fri, 07 Nov 2014 12:57:14 +0000",Have you ever heard of Amy Cuddy? In addition ...,Why Writers Should Care About Their Posture


# text processing

In [90]:
# remove HTML tags
# https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

df['text'] = df['text'].apply(strip_tags)

,cats,pubdate,text,title
0,"[Content Marketing, Tech Comm, Blog]","Thu, 20 Nov 2014 12:49:29 +0000",Imagine that you’ve just written what you beli...,10 Questions You’ve Got to Ask Yourself Before...
1,"[Content Marketing, Tech Comm, Love of Languag...","Mon, 17 Nov 2014 12:51:04 +0000",We recently shared a post containing part 1 of...,Problem Words that Give Writers Trouble (Part 2)
2,"[Content Marketing, Tech Comm, Love of Languag...","Thu, 13 Nov 2014 12:52:27 +0000",Even the best writers can get tripped up somet...,Problem Words that Give Writers Trouble (Part 1)
3,"[Content Marketing, Tech Comm, Love of Languag...","Mon, 10 Nov 2014 12:55:17 +0000",Part of being a good content marketer is being...,6 Great Free (or Nearly Free) Tools for Conten...
4,"[Content Marketing, Tech Comm, Love of Languag...","Fri, 07 Nov 2014 12:57:14 +0000",Have you ever heard of Amy Cuddy? In addition ...,Why Writers Should Care About Their Posture


In [165]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.strip()
    text = text.replace('’', "'")
    text = text.replace('“', '"')
    text = text.replace('”', '"')
    return text

df.text = df.text.apply(clean_text)

In [167]:
df.to_pickle('data/acrolinx_blog.pkl')

# how many sentences? & format for OpenNMT use

In [168]:
sents = []

for text in df.text:
    sents = sents + sent_tokenize(text)

In [169]:
len(sents)

10919

In [170]:
sents[len(sents)-10:len(sents)]

["And bad results aren't necessarily all bad news.",
 "They give you the means to identify areas of concern and allow you to redirect resources where they're needed most.",
 'A robust set of analytics is a critical ingredient in the content governance process.',
 'Analytics allow you to evaluate and then optimize your content and your content creation processes before you ever publish anything.',
 'On the path to enterprise content governance.',
 'Getting your arms around your content operations can seem overwhelming, but this four-step action plan will start you on the path to active content governance.',
 "When coupled with an AI-powered content governance platform, you'll be able to capture your strategy, establish your current position, align every content contributor with the goals you've set, and measure and track your success.",
 "It's really the best way to eliminate content chaos from your business once and for all.",
 '✔ Capture your strategy   ✔ Establish where you are today

In [171]:
with open('data/OpenNMT files/src-acrolinx.txt', 'w') as f:
    for sent in sents:
        if len(sent) > 5:
            f.write(' '.join(word_tokenize(sent)) + '\n')

# look at results after OpenNMT inf->f translation

In [2]:
with open('data/OpenNMT files/src-acrolinx.txt', 'r') as f:
    src = f.readlines()

with open('data/OpenNMT files/pred-acrolinx.txt', 'r') as f:
    pred = f.readlines()

In [3]:
src = [x.strip() for x in src]
pred = [x.strip() for x in pred]

In [4]:
assert len(src) == len(pred)

In [5]:
for idx in random.sample(range(len(src)), 10):
    print(src[idx])
    print(pred[idx])
    print()

Not only do you understand what the authors are trying to convey , you also get the sense that MailChimp is n't some big , soulless corporation .
Do you understand what the authors are trying to convey ? You also get the sense of soulless , soulless corporation .

It 's also where technical content faces the biggest challenges as a result of three converging trends : A rapid acceleration in software development cycles , including online launches Online hosting with embedded content , where bite-sized content produced at frequent intervals is replacing massive tome-sized documents produced over the course of many months An aspiration for localized content in multiple languages timed with product releases Let 's consider these trends , and their implications for technical content , individually .
In multiple languages , it 's technical content faces , the biggest challenges faces , a result of three months is content produced in software releases so I can understand technical content .



In [7]:
doccano = pd.DataFrame()

texts = []
for idx in range(len(src)):
    texts.append(src[idx] + ' ' + pred[idx])

In [9]:
doccano['text'] = texts
doccano.head()

,text
0,Imagine that you 've just written what you bel...
1,You 're ready to get it off your plate and sen...
2,"But wait just a second ! But , wait , a second !"
3,"Before you hit the publish button , are you po..."
4,"After all , you 've probably worked hard to cr..."


In [14]:
doccano.to_csv('data/doccano.csv')

# GYAFC corpus to doccano format

In [6]:
all_df = pd.read_pickle('data/rule_based_corrected_df.pkl')
all_df[80000:80005]

,Original,Target 0,Target 1,Target 2,Target 3,Category,Dataset
80000,"I do not have an answer for the question, but ...","I don't know about that question, but I really...",NaN,NaN,NaN,Entertainment_Music,train
80001,Do as you have been instructed.,Just do what everyone else said.,NaN,NaN,NaN,Entertainment_Music,train
80002,"They got to is first, go ahead and go there.",They beat me to it... yes go there,NaN,NaN,NaN,Entertainment_Music,train
80003,Did he not say he figured it out himself.,Didn't he say he came up with it himself,NaN,NaN,NaN,Entertainment_Music,train
80004,"Yes it was, and I enjoyed it.",Yes it was I enjoyed it,NaN,NaN,NaN,Entertainment_Music,train


In [8]:
texts = []
for idx, row in all_df.iterrows():
    text = row['Original']
    text = text + ' ' + str(row['Target 0'])
    text = text + ' ' + str(row['Target 1'])
    text = text + ' ' + str(row['Target 2'])
    text = text + ' ' + str(row['Target 3'])
    texts.append(text)

In [9]:
gyafc_docc = pd.DataFrame()
gyafc_docc['text'] = texts
gyafc_docc.head()

,text
0,I mean that you have to really be her friend. ...
1,Are you posing a rhetorical question? Sounds l...
2,Men pretend to love in order to have intercour...
3,I do not intend to be mean. I don't want to be...
4,I would estimate an average of 45% initially b...


In [11]:
gyafc_docc.to_csv('data/gyafc_docc.csv')

# post-doccano dataset analysis

In [90]:
with open('data/OpenNMT files/src-acrolinx.txt', 'r') as f:
    src = f.readlines()
src = [x.strip() for x in src]

In [91]:
ann = pd.read_json('data/acrolinxblog_annotated.json', lines=True)

In [92]:
ann['source'] = src

In [93]:
ann.head()

,annotations,id,meta,text,source
0,"[{'label': 1, 'end_offset': 22, 'user': 1, 'st...",2,{},"0,Imagine that you 've just written what you b...",Imagine that you 've just written what you bel...
1,"[{'label': 1, 'end_offset': 9, 'user': 1, 'sta...",3,{},"1,You 're ready to get it off your plate and s...",You 're ready to get it off your plate and sen...
2,[],4,{},"2,""But wait just a second ! But , wait , a sec...",But wait just a second !
3,"[{'label': 1, 'end_offset': 68, 'user': 1, 'st...",5,{},"3,""Before you hit the publish button , are you...","Before you hit the publish button , are you po..."
4,"[{'label': 1, 'end_offset': 22, 'user': 1, 'st...",6,{},"4,""After all , you 've probably worked hard to...","After all , you 've probably worked hard to cr..."


In [94]:
ex_text = ann['text'][0]
ex_start = ann['annotations'][0][0]['start_offset']
ex_end = ann['annotations'][0][0]['end_offset']

ex_text[ex_start:ex_end]

"you 've"

In [95]:
edits = []
tbd = []
num = 0

for idx, row in tqdm(ann.iterrows(), total=len(ann)):
    if row['annotations'] == []:
        edits.append(None)
    else:
        current_edits = []        
        # count number of annotations made
        num += len(row['annotations'])
        
        # split annotations into informal and formal changes
        inform_ann = []
        formal_ann = []
        for note in row['annotations']:
            if note['label'] == 1:
                inform_ann.append(note)
            elif note['label'] == 2:
                formal_ann.append(note)
                
        # set aside some to deal with later
        if len(inform_ann) != len(formal_ann):
            tbd.append((idx, row['annotations']))
            edits.append('None')
            continue
        elif inform_ann[-1]['end_offset'] > formal_ann[0]['start_offset']:
            tbd.append((idx, row['annotations']))
            edits.append('None')
            continue
            
        assert len(inform_ann) == len(formal_ann)
        for x in range(len(inform_ann)): # pairs of edits
            inform = row['text'][inform_ann[x]['start_offset']:inform_ann[x]['end_offset']]
            formal = row['text'][formal_ann[x]['start_offset']:formal_ann[x]['end_offset']]
            current_edits.append((inform.strip(), formal.strip()))
        edits.append(current_edits)
        
ann['edits'] = edits
print(num)
print(tbd)


9813
[(20, [{'label': 2, 'end_offset': 17, 'user': 1, 'start_offset': 4}, {'label': 1, 'end_offset': 161, 'user': 1, 'start_offset': 147}]), (32, [{'label': 2, 'end_offset': 33, 'user': 1, 'start_offset': 3}, {'label': 1, 'end_offset': 86, 'user': 1, 'start_offset': 57}]), (183, [{'label': 1, 'end_offset': 75, 'user': 1, 'start_offset': 57}, {'label': 1, 'end_offset': 200, 'user': 1, 'start_offset': 182}, {'label': 1, 'end_offset': 100, 'user': 1, 'start_offset': 89}, {'label': 2, 'end_offset': 228, 'user': 1, 'start_offset': 215}]), (265, [{'label': 2, 'end_offset': 12, 'user': 1, 'start_offset': 5}, {'label': 1, 'end_offset': 202, 'user': 1, 'start_offset': 197}]), (275, [{'label': 2, 'end_offset': 131, 'user': 1, 'start_offset': 77}, {'label': 1, 'end_offset': 237, 'user': 1, 'start_offset': 202}]), (547, [{'label': 2, 'end_offset': 18, 'user': 1, 'start_offset': 4}, {'label': 1, 'end_offset': 146, 'user': 1, 'start_offset': 128}]), (1511, [{'label': 1, 'end_offset': 17, 'user': 1,

In [112]:
len(ann)

10657

In [113]:
len(ann) - ann['edits'].isna().sum()

3817

In [137]:
# now: make formal base sentence with informal edits listed

new_formal = []
new_orig = []
new_repl = []

for idx, row in tqdm(ann.iterrows(), total=len(ann)):
    if type(row['edits']) == list:
        formal_sent = row['source']
        
        # retroactively create most formal sentence
        for item in row['edits']:
            if item[0] == item[1]:
                continue
            formal_sent = formal_sent.replace(item[0], item[1], 1)
            
        # add formal sentence and reversed edits
        for item in row['edits']:
            if item[0] == item[1]:
                continue
            elif len(item[0]) == 0:
                continue
            elif len(item[1]) == 0:
                continue
            elif item[0] == ' ' or item[1] == ' ':
                continue
            new_formal.append(formal_sent)
            new_orig.append(item[1])
            new_repl.append(item[0])
            
new_df = pd.DataFrame()
new_df['Sentence'] = new_formal
new_df['Original'] = new_orig
new_df['Replacement'] = new_repl

In [138]:
len(new_df)

4848

In [139]:
new_df.sample(50)

,Sentence,Original,Replacement
4215,It is important to remember the guidelines for...,you are,you 're
4822,"In order to calculate ROI , you can use this p...",In order to,To
2467,Only 2.3 percent of manufacturing companies ar...,that is,that 's
3569,"If you are a wordsmith woodpecker , you also u...",you are,you 're
497,It would mean that companies have a big opport...,It would mean,That means
162,How do you do that ?,How do,So how do
3733,It is also transformed the meaning of existing...,It is,It 's
99,"If you are a business writer , then you know t...",If you are a business writer,If you 're a business writer
2572,"It is quick and easy to consume , which is cri...",It is,It 's
763,"They want to know , they have , how many page ...",they have,they 've


In [140]:
new_df.to_pickle('data/acrolinx_blog_annotated_df.pkl')

In [141]:
c = Counter()

for idx, row in tqdm(new_df.iterrows(), total=len(new_df)):
    pair = row['Original'] + ' ' + row['Replacement']
    c[pair] += 1
    
c

Counter({'as well , too': 1,
         "I believe it is It 's": 1,
         'it is probably take': 1,
         'three 3': 1,
         'also , too': 1,
         "then what ca n't": 1,
         "you will also receive you 'll also receive": 1,
         'It is a A': 1,
         'I believe it means Alternate means': 1,
         'Well , my Well , sort of ... My': 1,
         'You should make make': 1,
         "large you are n't": 1,
         'may can': 1,
         "can not ca n't": 42,
         "you will be you 're going to be": 1,
         'Get So get': 2,
         'we we then': 1,
         "what is happening what 's happening": 1,
         "it will it 'll": 3,
         'Here in the United States Here in the US': 1,
         'Before But before': 2,
         "where is where 's": 1,
         'often often just': 2,
         "you are specifically doing you 're actually doing": 1,
         'going to gon na': 1,
         "It does not matter It does n't matter": 1,
         'I feel like Looks like

In [142]:
c.most_common()

[("you are you 're", 396),
 ("it is it 's", 310),
 ("It is It 's", 200),
 ("That is That 's", 156),
 ("do not do n't", 137),
 ("they are they 're", 120),
 ("that is that 's", 93),
 ("we are we 're", 90),
 ("you will you 'll", 87),
 ("is not is n't", 77),
 ("there is there 's", 60),
 ("you have you 've", 58),
 ("does not does n't", 53),
 ("we have we 've", 44),
 ("can not ca n't", 42),
 ("you have you 've got", 38),
 ("We are We 're", 38),
 ('. !', 32),
 ("are not are n't", 31),
 ("I have I 've", 26),
 ("They are They 're", 25),
 ("you would you 'd", 25),
 ("I am I 'm", 25),
 ("There is There 's", 25),
 ("It is it 's", 25),
 ("have not have n't", 24),
 ("Do not Do n't", 24),
 ("will not wo n't", 24),
 ('However , But', 24),
 ('may might', 22),
 ("You will You 'll", 20),
 ("We have We 've", 20),
 ("we will we 'll", 16),
 ('In the meantime Meanwhile', 16),
 ("they will they 'll", 16),
 ("Here is Here 's", 15),
 ("What is What 's", 15),
 ("they have they 've", 14),
 ("That is And that 's",

In [144]:
len(c)

1749

In [146]:
edit_classify_docc = pd.DataFrame()
edit_classify_docc['text'] = c.most_common()
edit_classify_docc.to_csv('data/edit_classify_docc.csv')

In [147]:
edit_classify_docc.head()

,text
0,"(you are you 're, 396)"
1,"(it is it 's, 310)"
2,"(It is It 's, 200)"
3,"(That is That 's, 156)"
4,"(do not do n't, 137)"
